In [1]:
import torch
import torchvision
from torchsummary import summary

import numpy as np
import sklearn
import sklearn.metrics

import matplotlib.pyplot as plt
from  tqdm.auto import tqdm

/Users/dell/anaconda3/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# check device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [87]:
import yaml
with open("config.yaml","r")as f:
    config = yaml.safe_load(f)
print(config)

{'model': 'test', 'lr': 0.1, 'batch_size': 3, 'epochs': 10}


# LOAD your data /data preparation

In [61]:
#Dataset preparation
import pandas as pd
class DataLoader_train(torch.utils.data.Dataset):
    def __init__(self,path_to_train):
        all_data = pd.read_csv(path_to_train)
        df = pd.DataFrame(all_data)
        df.head()
        self.target = df["target"]
        self.df_features = df.drop("target",axis=1)
    def __len__(self):
        return len(self.target)
    def __getitem__(self,ind):
        feature = torch.FloatTensor(self.df_features.iloc[ind]) # Convert to tensors
        target  = torch.tensor(self.target.iloc[ind])
        return feature,target


class DataLoader_test(torch.utils.data.Dataset):
    def __init__(self,path_to_test):
        test_data = pd.read_csv(path_to_test)
        df = pd.DataFrame(test_data)
        df.head()
        self.target = df["target"]
        self.df_features = df.drop("target",axis=1)
    def __len__(self):
        return len(self.target)
    def __getitem__(self,ind):
        feature = torch.FloatTensor(self.df_features.iloc[ind]) # Convert to tensors
        target  = torch.tensor(self.target.iloc[ind])
        return feature,target


In [62]:

train_val_dataset = DataLoader_train("all_data.csv")
train_dataset ,val_dataset = torch.utils.data.random_split(train_val_dataset, [0.8,0.2])
test_dataset = DataLoader_test("all_data.csv")

In [63]:
#Data loader

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=config["batch_size"],shuffle= True)
vac_loader = torch.utils.data.DataLoader(val_dataset,batch_size=config["batch_size"],shuffle= False)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=config["batch_size"],shuffle= False)

In [66]:
# Get first training sample & label pair
for i,data in enumerate(train_loader):
    feature,target = data
    print(feature.shape,target.shape)
    


torch.Size([15, 4]) torch.Size([15])


/var/folders/xv/tp46wlm94zz0n3x9szns5pjr0000gn/T/ipykernel_27376/746312654.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  feature = torch.FloatTensor(self.df_features.iloc[ind]) # Convert to tensors


# Define your nn 

In [83]:
class Network(torch.nn.Module):
    def __init__(self,input_size,output_size):
        super(Network,self).__init__()
        self.size = 1024
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size,input_size*2),
            torch.nn.BatchNorm1d(input_size*2),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(input_size*2,input_size),
            torch.nn.BatchNorm1d(input_size),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(input_size,output_size)
        )
    def forward(self,x):
        result = self.model(x)
        return result



In [84]:
input_size = feature.shape[1]
print(input_size)
output_size = 1
model = Network(input_size, output_size).to(device)
summary(model, (input_size,))

4
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [-1, 8]              40
       BatchNorm1d-2                    [-1, 8]              16
              ReLU-3                    [-1, 8]               0
           Dropout-4                    [-1, 8]               0
            Linear-5                    [-1, 4]              36
       BatchNorm1d-6                    [-1, 4]               8
              ReLU-7                    [-1, 4]               0
           Dropout-8                    [-1, 4]               0
            Linear-9                    [-1, 1]               5
Total params: 105
Trainable params: 105
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
-------------------------------------------------

In [89]:
#let us define the optimizer and loss
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr = config["lr"])
scheduler =  torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max= config["epochs"])


# well settled! next is to define train,eval process

In [90]:
import gc
torch.cuda.empty_cache()
gc.collect()

2230

In [ ]:
def train():
    